<div style="width: 100%; margin-left: auto; margin-right: auto;">
    <h2 style="text-align: center; margin-top: 1%; font-size:32px;">Avaliação de Desempenho - Atividade 7</h2>
</div>

**Aluno:** Igor Carvalho da Silva  
**Email:** igor.carvalho@icomp.ufam.edu.br  
**Matrícula:** 21951288  

---

## Dependências

In [1]:
!pip3 install numpy
!pip3 install pandas

In [2]:
import pandas as pd
import numpy as np

## Dataset
Leitura e tratamento dos dados

In [3]:
filename = 'Infocom'

In [4]:
df = pd.read_csv(filename + '.csv', sep=' ', header=None, names=['time','conn','src','dest','type'])
df

,time,conn,src,dest,type
0,20733,CONN,17,6,up
1,20821,CONN,16,26,up
2,20826,CONN,13,36,up
3,20868,CONN,17,32,up
4,20943,CONN,31,37,up
...,...,...,...,...,...
44913,274378,CONN,31,24,down
44914,274380,CONN,21,24,up
44915,274883,CONN,21,27,up
44916,274883,CONN,21,24,down


In [5]:
N = max(df['src']) + 1
adj_matrix = np.zeros((N,N))

In [6]:
for index, row in df.iterrows():
    adj_matrix[row.src][row.dest] = 1
    adj_matrix[row.dest][row.src] = 1

In [7]:
df_sorted = df.sort_values(['src','dest','type','time'])
df_full_time = pd.DataFrame(columns=['src', 'dest', 'time'])
for i in range(N):
    for j in range(i,len(adj_matrix[i])):
        if(adj_matrix[i][j] == 1):
            downs = list(
                df_sorted[(df_sorted.src == i) & (df_sorted.dest == j) & (df_sorted.type == 'down')]['time']
            )
            ups = list(
                df_sorted[(df_sorted.src == i) & (df_sorted.dest == j) & (df_sorted.type == 'up')]['time']
            )[:len(downs)]
            datas = pd.DataFrame([{'src': i, 'dest': j, 'time': down - up} for down, up in zip(downs, ups)])
            df_full_time = pd.concat([df_full_time, datas], ignore_index = True)
            
            downs = list(
                df_sorted[(df_sorted.src == j) & (df_sorted.dest == i) & (df_sorted.type == 'down')]['time']
            )
            ups = list(
                df_sorted[(df_sorted.src == j) & (df_sorted.dest == i) & (df_sorted.type == 'up')]['time']
            )[:len(downs)]
            datas = pd.DataFrame([{'src': i, 'dest': j, 'time': down - up} for down, up in zip(downs, ups)])
            df_full_time = pd.concat([df_full_time, datas], ignore_index = True)

## Número de conexões

In [8]:
total_connections = len(df[df['type'] == 'up'])
total_connections

22459

## Número de arestas

In [9]:
edges = 0
for i in range(N):
    edges = edges + sum(adj_matrix[i][i:])
edges

793.0

## Num médio de conexões por nó

In [10]:
nodes = len(set(list(df['src']) + list(df['dest'])))
connections_by_nodes = total_connections/nodes
connections_by_nodes

547.780487804878

## Conexões por minuto

In [11]:
connections_by_minute = total_connections/(max(df['time'])/60)
connections_by_minute

4.902231131063035

## Tempo total de conexão

In [12]:
total_time_connection = df_full_time['time'].sum()
total_time_connection

5204992

## Tempo médio de conexão

In [13]:
mean_time_connection = total_time_connection/nodes
mean_time_connection

126951.0243902439

## Grupo com mais nós conectados
(Sugestão: utilizar slots de 3h)

In [14]:
output_file = filename + '-group.txt'

In [15]:
slots = [list() for _ in range(df_full_time['time'].max() // 10800 + 1)]

In [16]:
for index, row in df_full_time.iterrows():
    obj = {'src': row.src, 'dest': row.dest, 'time': row.time}
    slots[row.time//10800].append(obj)

In [17]:
slots.sort(reverse=True, key=lambda x: len(x))

In [18]:
print(f'Tamanho do grupo com mais nós conectados: {len(slots[0])}')
print(f'O grupo será salvo no arquivo {output_file}')
with open(output_file, 'w') as file:
    for data in slots[0]:
        file.writelines(str(data) + '\n')

Tamanho do grupo com mais nós conectados: 22439
O grupo será salvo no arquivo Infocom-group.txt


## Média dos grupos com mais nós conectados
Como "os com mais nós conectados", foram escolhidos os 10 mais conectados, definido pela variável `n_rank`.

In [19]:
n_rank = 10

In [20]:
mean_slots = sum(list(map(lambda x: len(x), slots[:n_rank])))/10
mean_slots

2245.9

## Mediana dos grupos com mais nós conectados

In [21]:
top_slots = []
for slot in slots[:n_rank]:
    top_slots.append(list(map(lambda x: x['time'], slot)))

In [22]:
np.median(list(map(lambda x: np.median(x), top_slots)))

20622.5

## Número médio de nós conectados

In [23]:
def count_nodes(slot):
    m = np.zeros((N,N))
    for n in slot:
        m[n['src']][n['dest']] = 1
        m[n['dest']][n['src']] = 1
    edges = 0
    for i in range(N):
        edges = edges + sum(m[i][i:])
    return edges * 2

In [24]:
nodes_by_slot = list(map(count_nodes, slots))
mean_nodes_by_slots = sum(nodes_by_slot)/len(slots)
mean_nodes_by_slots

401.0

## Mediana do número de nós conectados

In [25]:
nodes_by_slot.sort()
median_nodes_by_slots = np.median(nodes_by_slot)
median_nodes_by_slots

8.0

## Maior grau dos nós por slot

In [26]:
def max_degree(slot):
    m = np.zeros((N,N))
    for n in slot:
        m[n['src']][n['dest']] = 1
        m[n['dest']][n['src']] = 1
    degrees = []
    for i in range(N):
        degrees.append(sum(m[i]))
    return max(degrees)

In [27]:
max_degrees_by_slot = list(map(max_degree, slots))
for i, degree in enumerate(max_degrees_by_slot):
    print(f'Slot: {i}, Maior Grau: {degree}')

Slot: 0, Maior Grau: 40.0
Slot: 1, Maior Grau: 2.0
Slot: 2, Maior Grau: 1.0
Slot: 3, Maior Grau: 1.0


## Grau médio dos nós por slot

In [28]:
def mean_degree(slot):
    m = np.zeros((N,N))
    for n in slot:
        m[n['src']][n['dest']] = 1
        m[n['dest']][n['src']] = 1
    degrees = []
    for i in range(N):
        degrees.append(sum(m[i]))
    nodes = count_nodes(slot)
    if(nodes != 0):
        return sum(degrees)/nodes
    return 0

In [29]:
mean_degrees_by_slot = list(map(mean_degree, slots))
for i, mean in enumerate(mean_degrees_by_slot):
    print(f'Slot: {i}, Média: {mean}')

Slot: 0, Média: 1.0
Slot: 1, Média: 1.0
Slot: 2, Média: 1.0
Slot: 3, Média: 1.0


## Criar uma lista dos contatos e suas durações
$(i,j,T_{ij})$ para todo $i,j \in N$

In [30]:
output_file = 'lista_infocom.txt'

In [31]:
print(f'A lista de contatos será salva no arquivo {output_file}')
df_full_time.to_csv(output_file, sep=' ', index=False)

A lista de contatos será salva no arquivo lista_infocom.txt


## Densidade do trace
$D = \frac{2|E|}{|V|(|V|-1)}$

In [32]:
density = (2*edges)/(nodes*(nodes-1))
density

0.9670731707317073